#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [2]:
import pandas as pd
import pymongo
import datetime

In [3]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [4]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (6,8,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [5]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [6]:
df_ks['launched'] = pd.to_datetime(df_ks['launched'], errors='coerce')
df_ks = df_ks.dropna(subset=['launched'])
df_ks.rename(columns={'ID':'_id'}, inplace=True)
l_d =df_ks.to_dict(orient='records')

### Importer les données

In [7]:
from pymongo import MongoClient
client = MongoClient("mongo:27017")

In [8]:
db_ks = client.ks
collection_ks = db_ks['abdoudb']

In [9]:
collection_ks.insert_many(l_d)

BulkWriteError: batch op errors occurred

In [ ]:
collection_ks.find_one()

## Question 1  

In [10]:
req = collection_ks.find().limit(5).sort([("usd_pledged_real", -1)])
for test in req:
    print(test)

{'_id': 342886736, 'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler", 'category': 'Product Design', 'main_category': 'Design', 'currency': 'USD', 'deadline': '2014-08-30', 'goal': '50000.0', 'launched': datetime.datetime(2014, 7, 8, 10, 14, 37), 'pledged': '13285226.36', 'state': 'successful', 'backers': '62642', 'country': 'US', 'usd pledged': '13285226.36', 'usd_pledged_real': 13285226.36}
{'_id': 2103598555, 'name': 'Pebble 2, Time 2 + All-New Pebble Core', 'category': 'Product Design', 'main_category': 'Design', 'currency': 'USD', 'deadline': '2016-06-30', 'goal': '1000000.0', 'launched': datetime.datetime(2016, 5, 24, 15, 49, 52), 'pledged': '12779843.49', 'state': 'successful', 'backers': '66673', 'country': 'US', 'usd pledged': '12779843.49', 'usd_pledged_real': 12779843.49}
{'_id': 1033978702, 'name': 'OUYA: A New Kind of Video Game Console', 'category': 'Gaming Hardware', 'main_category': 'Games', 'currency': 'USD', 'deadline': '2012-08-09', 'goal': '950000.

## Question 2

In [11]:
req = collection_ks.find({"$expr":{"$gte":["$pledged", "$goal"]}})
req.count()

67244

## Question 3

In [18]:
req = collection_ks.aggregate([{"$group": {"_id" : "$category", "projects" : {"$sum" : 1}}}])
list(req)

[{'_id': 'Wearables', 'projects': 508},
 {'_id': 'Childrenswear', 'projects': 192},
 {'_id': "Farmer's Markets", 'projects': 175},
 {'_id': 'Live Games', 'projects': 394},
 {'_id': 'Public Art', 'projects': 1248},
 {'_id': 'Documentary', 'projects': 6498},
 {'_id': 'Comic Books', 'projects': 1032},
 {'_id': 'Design', 'projects': 1641},
 {'_id': 'Community Gardens', 'projects': 115},
 {'_id': 'Animation', 'projects': 1017},
 {'_id': 'DIY Electronics', 'projects': 358},
 {'_id': 'Punk', 'projects': 127},
 {'_id': 'Ceramics', 'projects': 128},
 {'_id': 'Photo', 'projects': 58},
 {'_id': 'Robots', 'projects': 237},
 {'_id': 'Typography', 'projects': 44},
 {'_id': 'Glass', 'projects': 51},
 {'_id': 'Jazz', 'projects': 733},
 {'_id': 'Quilts', 'projects': 32},
 {'_id': 'Performance Art', 'projects': 855},
 {'_id': 'Thrillers', 'projects': 302},
 {'_id': 'Graphic Design', 'projects': 765},
 {'_id': 'Shorts', 'projects': 4857},
 {'_id': 'Workshops', 'projects': 59},
 {'_id': 'Bacon', 'projects

## Question 4

In [13]:
date = datetime.datetime.strptime("2016-01-01",'%Y-%m-%d')
req = collection_ks.find({"$and":[{'country':'FR'}, {'launched':{'$lt':date}}]})
req.count()

330

## Question 5

In [17]:
req = collection_ks.find({"$and":[{'country':'US'}, {'usd_pledged_real':{'$gte':200000}}]})
next(req)

{'_id': 217543389,
 'name': 'The uKeg Pressurized Growler for Fresh Beer',
 'category': 'Drinks',
 'main_category': 'Food',
 'currency': 'USD',
 'deadline': '2014-12-08',
 'goal': 75000.0,
 'launched': datetime.datetime(2014, 10, 15, 6, 34, 48),
 'pledged': 1559525.68,
 'state': 'successful',
 'backers': 10293,
 'country': 'US',
 'usd pledged': 1559525.68,
 'usd_pledged_real': 1559525.68}

## Question 6 

In [27]:
collection_ks.create_index([("name", "text")])
req = collection_ks.find({"$text": {"$search": "sport"}})
list(req)

[{'_id': 1875366029,
  'name': 'Sport Smart. A New Genre of Sports TV. Sport Fans Unite!!!',
  'category': 'Webseries',
  'main_category': 'Film & Video',
  'currency': 'USD',
  'deadline': '2011-01-15',
  'goal': 4500.0,
  'launched': datetime.datetime(2010, 12, 16, 8, 40, 2),
  'pledged': 25.0,
  'state': 'failed',
  'backers': 1,
  'country': 'US',
  'usd pledged': 25.0,
  'usd_pledged_real': 25.0},
 {'_id': 1126822169,
  'name': 'Frey Sports App - We connect sports people.',
  'category': 'Apps',
  'main_category': 'Technology',
  'currency': 'DKK',
  'deadline': '2017-01-11',
  'goal': 25000.0,
  'launched': datetime.datetime(2016, 12, 12, 11, 30, 28),
  'pledged': 0.0,
  'state': 'failed',
  'backers': 0,
  'country': 'DK',
  'usd pledged': 0.0,
  'usd_pledged_real': 0.0},
 {'_id': 1081541783,
  'name': 'Daily Fantasy Sports | Sports Analytics Platform | DFS',
  'category': 'Web',
  'main_category': 'Technology',
  'currency': 'USD',
  'deadline': '2017-12-09',
  'goal': 33750.0,